In [1]:
from langchain_community.llms import LlamaCpp

from prompt_factory import (
    SYSTEM_GRAPH_GENERATOR_PROMPT,
    GRAPH_GENERATOR_PROMPT,
    SYSTEM_CODE_CORRECTOR_PROMPT,
    CODE_CORRECTOR_PROMPT
)
from agents import LLMAgent
from tester import TestModule
from utils import try_correction

In [2]:
llm  = LlamaCpp(model_path = "models/cpp_models/llama_3.1_FP16.gguf",n_ctx=2048,max_tokens= 2000 ,repeat_last_n=20,stop=["END CODE"],top_k=3,n_gpu_layers=33,best_of_n=2,verbose=0)

/Users/scafartogregory/Desktop/idea_extraction/idea_extraction_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3490: UserWarning: WARNING! repeat_last_n is not default parameter.
                repeat_last_n was transferred to model_kwargs.
                Please confirm that repeat_last_n is what you intended.
  if await self.run_code(code, result, async_=asy):
/Users/scafartogregory/Desktop/idea_extraction/idea_extraction_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3490: UserWarning: WARNING! best_of_n is not default parameter.
                best_of_n was transferred to model_kwargs.
                Please confirm that best_of_n is what you intended.
  if await self.run_code(code, result, async_=asy):


In [3]:
Code_Generator_Agent = LLMAgent(llm, SYSTEM_GRAPH_GENERATOR_PROMPT,GRAPH_GENERATOR_PROMPT)
Code_Corrector_Agent = LLMAgent(llm, SYSTEM_CODE_CORRECTOR_PROMPT,CODE_CORRECTOR_PROMPT)

In [4]:
import numpy as np 
import pandas as pd
n=10
data = {
    'ID': np.arange(1, n+1),
    'Name': [f'Employee_{i}' for i in range(1, n+1)],
    'Age': np.random.randint(22, 65, size=n),
    'Salary': np.random.randint(35000, 120000, size=n),
    'Department': np.random.choice(['IT', 'HR', 'Finance', 'Marketing'], size=n),
    'Start Date': pd.date_range('2020-01-01', periods=n, freq='M')
}

# Create the DataFrame
df = pd.DataFrame(data)

/var/folders/g9/w9q193415dqd_3g5fx11gt_40000gn/T/ipykernel_15204/3454204520.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  'Start Date': pd.date_range('2020-01-01', periods=n, freq='M')


In [10]:
out = Code_Generator_Agent(
    {
       "dataframe":  df,
       "graph" : "bar chart"
        
    }
)

In [7]:
tester =  TestModule()

In [8]:
output = tester.exec_code(out)

In [9]:
if output["return_code"] == 1 :
    try_correction(Code_Corrector_Agent,tester=tester,output=output,nb_try=3)

In [ ]:
{
        "ID": list(np.arange(1, n + 1)),
        "Name": [f"Employee_{i}" for i in range(1, n + 1)],
        "Age": list(np.random.randint(22, 65, size=n)),
        "Salary": list(np.random.randint(35000, 120000, size=n)),
        "Department": list(np.random.choice(["IT", "HR", "Finance", "Marketing"], size=n)),
        "Start Date": list(pd.date_range("2020-01-01", periods=n, freq="M")),
    }

: 